In [ ]:
import json
import os
import random
import re
import time
from html import unescape
from queue import Queue
from urllib.request import urlretrieve

import openai
import pdfkit
import pyautogui
import PyPDF2
import requests
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


class Project:
    def __init__(self, name, web_element, link, link_element):
        self.name = name
        self.web_element = web_element
        self.link = link
        self.link_element = link_element

    def click(self):
        """Clicks the web element of the project."""
        self.web_element.click()

    def click_link(self):
        """Clicks the link element associated with the project."""
        self.link_element.click()


class FavProjects:
    def __init__(self, driver):
        self.driver = driver
        self.projects = self._get_favorite_projects()

    def _get_favorite_projects(self):
        """Private method to capture the web elements of each favorite project, their names, and their links."""
        project_xpath = '//div[@class="mt-8 mb-16 mx-12"]/ul[@class="list-none grid gap-y-5 mt-4"]/li'
        project_elements = self.driver.find_elements(By.XPATH, project_xpath)

        projects_list = []

        for project in project_elements:
            link_element = project.find_element(By.XPATH, './/a[@target="_blank"]')
            project_name = project.text.split("\n")[
                0
            ]  # Assuming the project name is the first line of text within the element

            proj = Project(project_name, project, link_element.get_attribute("href"), link_element)
            projects_list.append(proj)

        return projects_list


class Section:
    def __init__(self, combined_title, web_element, href):
        self.name = combined_title
        self.web_element = web_element
        self.href = href

    def __str__(self):
        return self.name


def attempt_click_image(image_name, attempts, confidence=0.7, grayscale=True):
    """Attempt to locate and click on an image using pyautogui for a specified number of times."""
    for _ in range(attempts):
        location = pyautogui.locateCenterOnScreen(image_name, confidence=confidence, grayscale=grayscale)
        if location:
            pyautogui.click(location)
            return True
        time.sleep(1)  # Delay for 1 second before the next attempt
    return False


def attempt_save_link_as(max_attempts, destination_path):
    """
    Attempts to find the "Save Link As" image on screen and triggers download.

    Args:
    - max_attempts: Number of attempts to find the "Save Link As" image.
    - save_directory: Directory path where the link content should be saved.

    Returns:
    - True if the "Save Link As" image is found and clicked, False otherwise.
    """

    attempts = 0
    while attempts < max_attempts:
        save_location = pyautogui.locateCenterOnScreen("save_link.png", confidence=0.7, grayscale=True)
        if save_location:
            download_file_via_save_as(save_location, destination_path)
            return True
        attempts += 1
        time.sleep(1)  # Wait a bit before trying again

    print("Failed to find the 'Save Link As' image after maximum attempts.")
    return False


def attempt_save_link_as_for_zip(max_attempts, save_directory):
    """
    Attempts to find the "Save Link As" image on screen and triggers download.

    Args:
    - max_attempts: Number of attempts to find the "Save Link As" image.
    - save_directory: Directory path where the link content should be saved.

    Returns:
    - True if the "Save Link As" image is found and clicked, False otherwise.
    """

    attempts = 0
    while attempts < max_attempts:
        save_location = pyautogui.locateCenterOnScreen("save_link.png", confidence=0.7, grayscale=True)
        if save_location:
            pyautogui.click(save_location)
            time.sleep(4)
            handle_zip_files(save_directory)
            return True
        attempts += 1
        time.sleep(1)  # Wait a bit before trying again

    print("Failed to find the 'Save Link As' image after maximum attempts.")
    return False


def attempt_to_click_location(location):
    """
    Attempt to click on the given location.

    Args:
    - location: The location to click on.

    Returns:
    - True if successfully clicked, otherwise False.
    """
    if location:
        pyautogui.click(location)
        return True
    return False


def attempt_to_find_button(region, max_attempts, timeout_per_attempt, driver):
    """
    Attempt to locate and click the button within the given region.
    
    Args:
    - region (tuple): The coordinates of the region to search within.
    - max_attempts (int): The maximum number of attempts to find the button.
    - timeout_per_attempt (int): Time to wait in seconds before each retry.
    
    Returns:
    - bool: True if button is found and clicked, False otherwise.
    """
    seconds = 0
    attempts = 0
    while attempts < max_attempts:
        center_location = wait_and_locate_image(
            "download_button_2.png", region=region, confidence=0.7, timeout=timeout_per_attempt
        )
        if attempt_to_click_location(center_location):
            print("DEBUG: center_location =", center_location)
            return True
        if seconds == 5:
            play_video_if_paused(driver)

        time.sleep(1)  # Wait for 1 second before the next check
        attempts += 1
        seconds += 1
    return False


def check_zip_file(link):
    """
    Determines if the given link points to a ZIP file.
    
    Args:
    - link: A Selenium WebElement representing a download link.
    
    Returns:
    - A boolean indicating whether the link points to a ZIP file.
    """
    href_value = link.get_attribute("href")
    is_zip_file = href_value and href_value.endswith(".zip")

    data_src = link.get_attribute("data-src")
    data_src_is_zip = "zip.svg" in (data_src or "")

    return is_zip_file or data_src_is_zip


def click_1080_quality():
    """Attempt to click on 1080 video quality."""
    return attempt_click_image("1080.png", 2, confidence=0.7)


def click_1920_quality():
    """Attempt to click on 1920 video quality."""
    return attempt_click_image("1920.png", 2, confidence=0.7)


def click_720_quality():
    """Attempt to click on 720 video quality."""
    return attempt_click_image("720.png", 2, confidence=0.8)


def click_directory():
    """Click on the file name input area."""
    pyautogui.moveTo(234, 79)
    pyautogui.click()


def click_download():
    """Click the download button."""
    pyautogui.moveTo(881, 1047)
    pyautogui.click()


def click_file_name():
    """Click on the file name input area."""
    time.sleep(1)
    pyautogui.moveTo(592, 691)
    pyautogui.click()


def click_file_path():
    """Click on the file path input area."""
    pyautogui.moveTo(817, 838)
    pyautogui.click()


def click_on_first_video(video_elements):
    """
    Click on the first video in the list of video elements.

    Args:
    - video_elements: List of video elements.

    Returns:
    - None
    """
    if video_elements:
        video_elements[0].click()
        time.sleep(4)
        print("Clicked on the first video!")
    else:
        print("No video elements found!")


def click_save():
    """Click on the file name input area."""
    pyautogui.moveTo(1986, 1447)
    pyautogui.click()


def click_video_logo():
    """Attempt to click on the video logo."""
    return attempt_click_image("video_logo.png", 2, confidence=0.7)


def click_video_quality():
    """Attempt to locate and click on 1080p or 720p video quality."""
    global driver
    if video_contains_s3_bucket(driver):
        if click_video_logo():
            return

    if click_1920_quality():
        return

    if click_1080_quality():
        return

    if click_720_quality():
        return

    if click_video_logo():
        return

    print("Neither 1920, 1080p, 720p, nor video logo found!")


def create_project_folder(project_name, original_path):
    """Create a directory for the project if it doesn't exist."""
    folder_name = "".join(e for e in project_name if e.isalnum() or e in [" ", "_"]).rstrip()
    project_path = os.path.join(original_path, folder_name)
    if not os.path.exists(project_path):
        os.makedirs(project_path)
    return project_path


def download_file_via_save_as(save_location, destination_path):
    """
    Automates the process to download a file by clicking the "Save As" option using pyautogui.

    Args:
    - save_location: Location (x, y) of the "Save As" option.
    - save_directory: Directory path where the link content should be saved.

    Returns:
    - None
    """
    pyautogui.click(save_location)
    time.sleep(6)

    click_directory()  # Assuming this function is already defined

    # Wait for the "Save As" dialog, then type the directory path and press Enter
    time.sleep(3)  # Pause before typing the save path
    pyautogui.write(destination_path)
    pyautogui.press("enter")

    time.sleep(4)
    click_save()  # Assuming this function is already defined

    time.sleep(6)


def download_video(video_title, file_path):
    # Add an explicit wait here
    time.sleep(2)

    click_file_name()
    write_file_name(video_title)
    click_file_path()

    write_file_path(file_path)
    click_download()


def extract_leading_number(s):
    """Extract the leading number from a string."""
    # Split the string at the first occurrence of "."
    parts = s.split(".", 1)

    # Check if the first part is a digit
    if parts[0].isdigit():
        return int(parts[0])
    else:
        return None  # or raise an error, or handle it however you like


def extract_longest_section_from_driver(driver):
    parent_elements = driver.find_elements(
        By.XPATH,
        "/html/body/div[2]/div[2]/ul/li/ul/li/span[@class='d-flex justify-content-between align-items-baseline']",
    )

    max_time = 0  # Keep track of the maximum time in minutes
    longest_section = None  # Store the section object with the longest time

    for parent_elem in parent_elements:
        # Extract section and time's outerHTML
        section_html = parent_elem.find_element(By.XPATH, "./a[span[@class='title']]").get_attribute("outerHTML")
        time_html = parent_elem.find_element(By.XPATH, "./span[@class='time text-light-black']").get_attribute(
            "outerHTML"
        )
        href_value = parent_elem.find_element(By.XPATH, "./a[span[@class='title']]").get_attribute("href")

        # Extract relevant information from the HTML strings
        video_index = unescape(section_html.split('data-current-video-index-no="')[1].split('"')[0])
        video_name = unescape(section_html.split('data-current-video-name="')[1].split('"')[0])
        video_time = unescape(time_html.split('<span class="time text-light-black">')[1].split("</span>")[0])

        # Parse time from the format (12m)
        time_in_minutes = int(re.search(r"(\d+)m", video_time).group(1))

        # Check if this time is greater than the current max_time
        if time_in_minutes > max_time:
            max_time = time_in_minutes
            combined_title = f"{video_index}. {video_name} ({video_time})"
            longest_section = Section(combined_title, parent_elem, href_value)

    return longest_section.web_element


def extract_prerequisite_sections(li, counter, pre_counter, previous_video_index):
    prereq_sections = []
    all_sections = []
    letter_ascii = ord("a")  # Initialize ASCII value for 'a'

    prerequisite_tabs = li.find_elements(By.XPATH, ".//a[contains(@class, 'prerequisite-tab')]")
    all_sections.append(Section(f"{pre_counter}. prerequisite section", prerequisite_tabs[0], None))
    pre_counter += 1

    video_links = li.find_elements(By.XPATH, ".//ol[contains(@class, 'reference-video-tabs')]/li/a")
    for link in video_links:
        href_value = link.get_attribute("href")
        video_name = link.get_attribute("data-current-video-name")
        video_index = link.get_attribute("data-current-video-index-no")
        if not video_index:
            counter += 1
            video_index = str(counter)

        combined_title = format_prerequisite_title(video_index, video_name, previous_video_index, letter_ascii)
        prereq_sections.append(Section(combined_title, link, href_value))
        all_sections.append(Section(combined_title, link, href_value))

        letter_ascii += 1
        previous_video_index = video_index

    return all_sections, prereq_sections, counter, pre_counter, previous_video_index


def extract_regular_video_sections(li, counter):
    try:
        parent_elem = li.find_element(By.XPATH, ".//span[@class='d-flex justify-content-between align-items-baseline']")
        section_html = parent_elem.find_element(By.XPATH, "./a[span[@class='title']]").get_attribute("outerHTML")
        time_html = parent_elem.find_element(By.XPATH, "./span[@class='time text-light-black']").get_attribute(
            "outerHTML"
        )
        href_value = parent_elem.find_element(By.XPATH, "./a[span[@class='title']]").get_attribute("href")

        combined_title = format_regular_video_title(section_html, time_html, counter)
        section = Section(combined_title, parent_elem, href_value)
        return [section], counter + 1

    except NoSuchElementException:
        return [], counter


def extract_sections_from_driver(driver):
    parent_list_items = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/ul/li/ul/li")
    print(f"Total list items found: {len(parent_list_items)}")

    all_sections = []
    counter = 0
    pre_counter = 1
    previous_video_index = None

    for li in parent_list_items:
        prerequisite_tabs = li.find_elements(By.XPATH, ".//a[contains(@class, 'prerequisite-tab')]")
        if prerequisite_tabs:
            sections, _, counter, pre_counter, previous_video_index = extract_prerequisite_sections(
                li, counter, pre_counter, previous_video_index
            )
            all_sections.extend(sections)
            continue

        sections, counter = extract_regular_video_sections(li, counter)
        all_sections.extend(sections)

    return all_sections


def file_exists(destination_path, section_name):
    path_to_check = os.path.join(destination_path, f"{remove_windows_special_chars(section_name)}.mp4")
    return os.path.exists(path_to_check)


def find_center(timeout_per_attempt=60):
    """Repeatedly tries to locate the center of an image until timeout."""
    end_time = time.time() + timeout_per_attempt
    region = (1836, 115, 84, 965)
    while time.time() < end_time:
        center_location = pyautogui.locateCenterOnScreen("download_button_2.png", region=region, confidence=0.8)
        if center_location:
            return center_location
        time.sleep(0.1)  # Poll every 0.1 second during the timeout interval
    return None


def find_visible_video_elements(driver):
    """
    Find and return all visible video elements on the page.

    Args:
    - driver: Selenium WebDriver instance.

    Returns:
    - List of visible video elements.
    """
    video_elements = driver.find_elements(By.XPATH, "//label[contains(@class, 'video-reference-link')]")
    return [video for video in video_elements if video.is_displayed()]


def format_prerequisite_title(video_index, video_name, previous_video_index, letter_ascii):
    if video_index != previous_video_index:
        letter_ascii = ord("a")
    letter = chr(letter_ascii)
    return f"{video_index}. ({letter}) prerequisite - {video_name}"


def format_regular_video_title(section_html, time_html, counter):
    try:
        video_index = unescape(section_html.split('data-current-video-index-no="')[1].split('"')[0])
    except IndexError:
        counter += 1
        video_index = str(counter)
    video_name = unescape(section_html.split('data-current-video-name="')[1].split('"')[0])
    video_time = unescape(time_html.split('<span class="time text-light-black">')[1].split("</span>")[0])
    return f"{video_index}. {video_name} ({video_time})"


def go_to_another_section_and_return(current_section):
    all_sections = extract_sections_from_driver(driver)

    # Appending the section that matches the current section name.
    for section in all_sections:
        if section.name == current_section.name:
            navigate_to_section(section)
            break  # Once found, exit the loop.

    # Make sure we choose an interactable random section
    all_sections.remove(section)
    interactable_found = False
    while not interactable_found:
        random_section = random.choice(all_sections)
        try:
            random_section.web_element.click()  # Try clicking the web_element
            interactable_found = True  # If it succeeds, mark it as interactable
            navigate_to_section(random_section)
            break
        except ElementNotInteractableException:
            continue  # If not interactable, loop again to choose another section

    refresh_page()


def handle_download(driver, link, is_zip, max_attempts, destination_path):
    """
    Handles the download process for a link.
    
    Args:
    - driver: Initialized Selenium WebDriver instance.
    - link: A Selenium WebElement representing a download link.
    - is_zip: A boolean indicating whether the link is for a ZIP file.
    - max_attempts: Maximum attempts for the save action.
    - destination_path: Directory path where the file should be saved.
    
    Returns:
    - None
    """
    print(f"File Name: {link.text} | Is ZIP File: {is_zip}")

    ActionChains(driver).context_click(link).perform()
    time.sleep(2)  # Allow time for context menu to appear

    if is_zip:
        print("This is a ZIP file")
        if attempt_save_link_as_for_zip(max_attempts, destination_path):
            return

    if not attempt_save_link_as(max_attempts, destination_path):
        return


def handle_prerequisite_section(driver, action, section, destination_path):
    """Handle the actions for a prerequisite section."""
    library = extract_sections_from_driver(driver)
    element = return_element(library, section)
    action.move_to_element(element.web_element).click().perform()
    time.sleep(6)


def handle_regular_section(driver, action, section, destination_path):
    """Handle the actions for a regular section."""
    print("processing", section.name)
    if file_exists(destination_path, section.name):
        print("file exists", section.name)
        return

    library = extract_sections_from_driver(driver)
    element = return_element(library, section)
    print("clicking on", element.name)
    action.move_to_element(element.web_element).click().perform()
    time.sleep(7)

    if video_contains_s3_bucket(driver):
        refresh_page()

    process_section(section)


def handle_zip_files(directory):
    """
    This function will handle the specific operations required for ZIP files.
    Implement the actions you want to perform for ZIP files.
    """
    pyautogui.moveTo(689, 859)
    pyautogui.click()

    pyautogui.write(directory)

    pyautogui.moveTo(876, 1099)
    pyautogui.click()


def navigate_to_section(section):
    action.move_to_element(section.web_element).click().perform()
    time.sleep(6)


def play_video_if_paused(driver):
    """
    If a video is present on the current page and paused, play it.

    Args:
    - driver (webdriver instance): The Selenium webdriver instance.
    """

    try:
        # Attempt to locate the video element
        video = driver.find_element(By.TAG_NAME, "video")

        # Print the paused state
        paused_state = video.get_attribute("paused")
        print(paused_state)

        # Check if the video is paused
        if paused_state == "true" or paused_state is True:  # Handle both string and boolean possibilities
            # Locate the play button using its XPath
            play_button = driver.find_element(By.XPATH, '//*[@id="videoJsPlayer"]/button')
            # Click the play button to play the video
            play_button.click()

    except NoSuchElementException:
        # Handle the case where the video element isn't present
        print("No video element found on the page.")


def process_section(section):
    """Process a given section."""
    global counter, destination_path, driver  # Use the global counter variable
    print("here")
    time.sleep(2)

    if wait_and_click_download_button(section, driver):
        video_logo = click_video_quality()
    else:
        print("Download button not found!")
        return

    download_video(section.name, destination_path)
    print(destination_path, section.name)
    wait_for_file(os.path.join(destination_path, f"{remove_windows_special_chars(section.name)}.mp4"))
    print(os.path.join(destination_path, f"{remove_windows_special_chars(section.name)}.mp4"))

    counter += 1


def process_section_for_download(driver, action, section, destination_path):
    """Process each section and take necessary actions."""
    if "prerequisite section" in section.name.lower():
        handle_prerequisite_section(driver, action, section, destination_path)
    else:
        handle_regular_section(driver, action, section, destination_path)


def refresh_page():
    pyautogui.moveTo(128, 83)
    pyautogui.click()
    time.sleep(6)


def remove_windows_special_chars(string, replacement="-"):
    """Sanitize a file path by removing or replacing Windows reserved characters."""
    reserved_chars = ["<", ">", ":", '"', "/", "\\", "|", "?", "*"]
    for char in reserved_chars:
        string = string.replace(char, replacement)
    return string


def return_element(lst, section):
    for sec in lst:
        if sec.name == section.name:
            return sec


def save_files_from_webpage(driver, destination_path):
    """
    Download files from a webpage to a specific directory on your system.

    Args:
    - driver: Selenium WebDriver instance.
    - save_path: Directory where you want to save the files.
    """
    scroll_until_button_found(driver)
    save_links_as(driver, destination_path)


def save_links_as(driver, destination_path, max_attempts=5):
    download_link_xpath = "//a[@class='file-download-href']"
    scroll_until_element_found(driver, download_link_xpath)

    # Locate all download links
    download_links = driver.find_elements(By.XPATH, download_link_xpath)
    print(len(download_links))
    for link in download_links:
        is_zip = check_zip_file(link)
        handle_download(driver, link, is_zip, max_attempts, destination_path)


def scroll_until_button_found(driver, pixels=300):
    """Smoothly scrolls down on the webpage in increments until the download button is found or the end of the page is reached."""

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        try:
            download_button = driver.find_element(By.XPATH, "//a[@id='download-tab']")
            download_button.click()
            time.sleep(2)
            break
        except:
            driver.execute_script(f"window.scrollBy(0, {pixels});")
            time.sleep(2)  # Let the page load after scrolling

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("Reached end of page without finding the download button.")
                break

            last_height = new_height


def scroll_until_element_found(driver, element_xpath, pixels=300):
    """Smoothly scrolls down on the webpage in increments until the specified element is found or the end of the page is reached."""

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        try:
            # Try to find the specified element and check if it's clickable
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, element_xpath)))
            break
        except:
            # If not found or not clickable, continue scrolling
            driver.execute_script(f"window.scrollBy(0, {pixels});")
            time.sleep(2)  # Let the page load after scrolling

            # Check the new scroll height after the scroll
            new_height = driver.execute_script("return document.body.scrollHeight")

            # If the new height is the same as the last height, we've reached the bottom
            if new_height == last_height:
                print(f"Reached end of page without finding the clickable element with XPath: {element_xpath}")
                break

            last_height = new_height


def switch_to_new_tab(driver):
    """Switches to a newly opened tab."""
    main_window = driver.current_window_handle  # Store the main window handle
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
    driver.switch_to.window(driver.window_handles[-1])  # Switch to the new tab
    time.sleep(3)
    return main_window


def video_contains_s3_bucket(driver):
    video_element = driver.find_element(By.XPATH, '//video[@id="videoJsPlayer_html5_api"]')
    video_url = video_element.get_attribute("src")
    if "s3.amazonaws" in video_url:
        return True
    return False


def wait_and_click_download_button(current_section, driver, max_attempts=15, timeout_per_attempt=1):
    """Continuously checks for the download button every second and clicks its center once found."""
    time.sleep(5)  # Initial wait time
    region = (1846, 357, 336, 59)

    if attempt_to_find_button(region, max_attempts, timeout_per_attempt, driver):
        return True

    # If the button isn't found after the initial attempts, navigate to another section and retry
    print("Download button not found. Navigating to another section.")
    go_to_another_section_and_return(current_section)

    # A brief pause to account for possible loading or animations when navigating
    time.sleep(7)

    if attempt_to_find_button(region, max_attempts, timeout_per_attempt, driver):
        return True

    # If the function reaches here, the button wasn't found even after navigating to another section
    return False


def wait_and_locate_image(image_name, region=None, confidence=0.7, timeout=1):
    """
    Wait for the specified timeout and try to locate an image.

    Args:
    - image_name: The name of the image file to locate.
    - region: The region of the screen to search for the image.
    - confidence: The confidence level for the image search.
    - timeout: The time to keep searching for the image.

    Returns:
    - The center location of the image if found, otherwise None.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        center_location = pyautogui.locateCenterOnScreen(image_name, region=region, confidence=confidence)
        if center_location:
            return center_location
        time.sleep(0.1)  # Poll every 0.1 second during the timeout interval
    return None


def wait_for_file(filepath):
    """Wait for a file to appear in the directory."""
    while not os.path.exists(filepath):
        time.sleep(1)


def write_file_name(video_title):
    """Type the video title with the .mp4 extension into the input field."""
    pyautogui.write(f"{video_title}.mp4")


def write_file_path(file_path):
    """Type the provided file path into the input field."""
    pyautogui.write(file_path)
    
    
if __name__ == "__main__":
    options = webdriver.ChromeOptions()
    profile_path = ""  # insert path here for user profile
    options.add_argument("--user-data-dir=" + profile_path)
    options.add_argument("--profile-directory=DefaultCopy")

    driver = webdriver.Chrome(options=options)
    driver.get("")
    driver.maximize_window()
    time.sleep(3)

    destination_path = r""
    original_path = r""


    # Main code starts here:
    favorite_projects = FavProjects(driver)
    for project in favorite_projects.projects:
        print(f"Processing project: {project.name}")
        action = ActionChains(driver)

        destination_path = create_project_folder(project.name, original_path)
        time.sleep(2)

        project.click_link()
        main_window = switch_to_new_tab(driver)

        all_sections = extract_sections_from_driver(driver)
        time.sleep(5)

        counter = 0
        for section in all_sections:
            process_section_for_download(driver, action, section, destination_path)
            print(section.name)

        # Extract the longest section
        longest_displayed_section = extract_longest_section_from_driver(driver)
        action.move_to_element(longest_displayed_section).click().perform()
        time.sleep(4)

        save_files_from_webpage(driver, destination_path)
        driver.close()  # Close the current tab
        driver.switch_to.window(main_window)  # Switch back to the main/original tab